# Creating question variants
EDSL comes with a variety of features for efficiently generating different versions of questions in surveys. This notebook demonstrates methods for doing this with `Scenario` objects.

## What is a `Scenario`?
A `Scenario` is a dictionary of one or more key/value pairs representing data or content to be added to questions; a `ScenarioList` is a list of `Scenario` objects. 
Scenario keys are used as question parameters that get replaced with the values when the scenarios are added to the questions, allowing you to create variants of questions efficiently.
For example:

```
from edsl import QuestionFreeText, ScenarioList, Scenario

q = QuestionFreeText(
    question_name = "favorite",
    question_text = "What is your favorite {{ thing }}?"
)

s = ScenarioList(
    Scenario({"thing": t}) for t in ["flower", "pizza topping", "chatbot"]
)
```

## Using scenarios
Scenarios can be added to questions when constructing a survey or when running it.
Functionally, the same question context is delivered to agents and models whether they are added during or after survey construction. 
The difference is how the information is arranged in the results that are generated by the models. 

## Methods

### Adding scenarios at survey construction: `loop`
Each question type (`QuestionMultipleChoice`, `QuestionFreeText`, etc.) has a `loop()` method that generates a copy of the question for each scenario in a `ScenarioList` that is passed to it, returning a list of the questions that are generated. 
The `loop()` method is used when survey questions are being constructed. 
The typical workflow is:

* Construct a (single) `Question` with one or more parameters
* Construct a `ScenarioList`
* Call the `loop()` method on the question and pass it the scenario list
* Pass the list of the questions to a `Survey`

From the example above this looks like:
```
from edsl import Survey

questions = q.loop(s)
survey = Survey(questions)
results = survey.run()
```

When the survey is run, the results that are generated will include columns for each question and answer; there are no `scenario` columns in the results (unless scenarios are *also* added when the survey is run).

### Running a survey with scenarios: `by` 
Scenarios can also be passed to a question or survey at the time that it is run. 
This is done by calling the `by()` method on a survey, passing it the scenarios, and then calling the `run()` method.
The typical workflow is:

* Construct a question (or survey of multiple questions) with one or more parameters
* Construct scenarios
* Call the `by()` method on the question or survey and pass it the scenarios
* Call the `run()` method to administer the question or survey

From the example above this looks like:
```
results = q.by(s).run()
```

(If any agents or models have been created and specified, they would also be added in separated `by()` calls. 
See details on [designing agents](https://docs.expectedparrot.com/en/latest/agents.html) and [selecting language models](https://docs.expectedparrot.com/en/latest/language_models.html).)

## Example: Looping a question with scenarios
The `loop()` method is called on a `Question` object, and takes a `ScenarioList` of values to be inserted in copies of the question. 
We can optionally use the scenario key in the question name as well (so long as it is Pythonic); otherwise, unique identifiers are added to the original question name.

We start by constructing a question that takes a parameter:

In [1]:
from edsl import QuestionFreeText

q = QuestionFreeText(
    question_name = "features",
    question_text = "What are the features of this sailboat model: {{ sailboat_model }}"
)

Next we create a scenario list to pass to the `loop()` method. 
EDSL comes with many methods for generating scenarios from different data sources, such as PDFs, CSVs, docs, tables, images, etc.
For example, we can use the `from_list()` method to construct a scenario list from a list. 
[Learn about other methods for generating scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html).

In [2]:
from edsl import ScenarioList

s = ScenarioList.from_list("sailboat_model", ['Laser', 'Sunfish', 'Optimist', 'Finn'])
s

ScenarioList([Scenario({'sailboat_model': 'Laser'}), Scenario({'sailboat_model': 'Sunfish'}), Scenario({'sailboat_model': 'Optimist'}), Scenario({'sailboat_model': 'Finn'})])

Next we call the `loop()` method with the scenario list to create a list of the copies of the question, and verify that formatted questions have been generated:

In [3]:
questions = q.loop(s)
questions

[Question('free_text', question_name = """features_0""", question_text = """What are the features of this sailboat model: Laser"""),
 Question('free_text', question_name = """features_1""", question_text = """What are the features of this sailboat model: Sunfish"""),
 Question('free_text', question_name = """features_2""", question_text = """What are the features of this sailboat model: Optimist"""),
 Question('free_text', question_name = """features_3""", question_text = """What are the features of this sailboat model: Finn""")]

We can pass the questions to a `Survey` and then run it:

In [4]:
from edsl import Survey

survey = Survey(questions)
results = survey.run()

We can check the columns of dataset of `Results` that have been generated, and see that there are sets of columns for each question identifiable by question name (but no `scenario` columns):

In [5]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.features_0',
 'answer.features_1',
 'answer.features_2',
 'answer.features_3',
 'generated_tokens.features_0_generated_tokens',
 'generated_tokens.features_1_generated_tokens',
 'generated_tokens.features_2_generated_tokens',
 'generated_tokens.features_3_generated_tokens',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.features_0_system_prompt',
 'prompt.features_0_user_prompt',
 'prompt.features_1_system_prompt',
 'prompt.features_1_user_prompt',
 'prompt.features_2_system_prompt',
 'prompt.features_2_user_prompt',
 'prompt.features_3_system_prompt',
 'prompt.features_3_user_prompt',
 'question_options.features_0_question_options',
 'question_options.features_1_question_options',
 'question_options.features_2_question_options',
 'question_options.features_3_question_options',
 'question_text.fea

We can access [built-in methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html), e.g., printing a table:

In [6]:
results.select("answer.*").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                     ┃ answer                    ┃ answer                     ┃ answer                    ┃
┃ .features_1                ┃ .features_0               ┃ .features_3                ┃ .features_2               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ The Sunfish is a popular   │ The Laser is a popular    │ The Finn is a              │ The Optimist, often       │
│ sailboat that is known for │ one-design class of small │ single-handed, cat-rigged  │ referred to as an "Opti," │
│ its simplicity and ease of │ sailing dinghy. Here are  │ Olympic class sailboat     │ "Oppie," or "bathtub," is │
│ use, making it a favorite  │ some of its key features: │ that has been a staple of  │ a small, single-handed    │
│ among beginners and        │                           │ Olympic sailing            │ sailing dinghy intended   │
│ experienced sailors alike. │ 1. **Design**: The Laser  │ competitions since its     │ for use by children up to │
│ Here are some of its key   │ is designed to be simple  │ introduction in the 1950s. │ the age of 15. It is one  │
│ features:                  │ and fast. It has a single │ Designed by Swedish canoe  │ of the most popular       │
│                            │ sail, a single hull, and  │ designer Rickard Sarby in  │ sailing dinghies in the   │
│ 1. **Size and Hull         │ is built for performance  │ 1949, the Finn has a rich  │ world, designed to be     │
│ Design**: The Sunfish has  │ and ease of use.          │ history and has been the   │ safe, simple, and stable  │
│ a very simple,             │                           │ boat of choice for many    │ while providing a         │
│ flat-bottomed hull design. │ 2. **Size and             │ legendary sailors.         │ challenging and           │
│ It is small and            │ Dimensions**:             │                            │ competitive environment   │
│ lightweight, typically     │    - Length: 4.23 meters  │ Here are some of the key   │ for more advanced young   │
│ about 13.9 feet (4.2       │ (13.9 feet)               │ features of the Finn       │ sailors. Here are some of │
│ meters) in length and 4.1  │    - Beam (width): 1.37   │ sailboat:                  │ the key features of the   │
│ feet (1.2 meters) in beam  │ meters (4.5 feet)         │                            │ Optimist sailboat model:  │
│ (width). The hull is       │    - Draft (with the      │ 1. **Hull Design**: The    │                           │
│ usually made from          │ centerboard down): 0.787  │ Finn features a hard-chine │ 1. Size and Hull:         │
│ fiberglass, which provides │ meters (2.6 feet)         │ hull with a distinctive,   │    - Length: 7 feet, 9    │
│ a good balance between     │                           │ classic shape that allows  │ inches (2.36 meters)      │
│ durability and weight.     │ 3. **Sail Area**:         │ for powerful upwind        │    - Beam (width): 3      │
│                            │    - Standard Rig: 7.06   │ performance. The hull is   │ feet, 8 inches (1.12      │
│ 2. **Sail**: The boat has  │ square meters (76 square  │ relatively heavy, which    │ meters)                   │
│ a lateen sail (a           │ feet)                     │ provides stability in      │    - Hull weight: roughly │
│ triangular sail), which is │    - Radial Rig: 5.76     │ various wind conditions.   │ 77 pounds (35 kilograms)  │
│ mounted to an un-stayed    │ square meters (62 square  │                            │    - The hull is          │
│ mast (a mast without       │ feet)                     │ 2. **Rigging**: The Finn   │ pram-shaped, which means  │
│ support wires or stays).   │    - 4.7 Rig: 4.7 square  │ is equipped with a single  │ it has a flat front or    │
│ The sail area is typically │ meters (50.6 square feet)

## Running a question with scenarios
If we instead want to add the scenarios to the question when it is run, we simply add them with the `by()` method. This will re-administer a question for each scenario:

In [7]:
results = q.by(s).run()

The results now include columns for the single question but with a separate row for each scenario:

In [8]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.features',
 'generated_tokens.features_generated_tokens',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.features_system_prompt',
 'prompt.features_user_prompt',
 'question_options.features_question_options',
 'question_text.features_question_text',
 'question_type.features_question_type',
 'raw_model_response.features_raw_model_response',
 'scenario.sailboat_model']

In [9]:
results.select("sailboat_model", "features").print(format="rich")  # results.select("scenario.*", "answer.*") is equivalent here

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario        ┃ answer                                                                                        ┃
┃ .sailboat_model ┃ .features                                                                                     ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Laser           │ The Laser is a popular one-design class of small sailing dinghy. Here are some of its key     │
│                 │ features:                                                                                     │
│                 │                                                                                               │
│                 │ 1. **Design**: The Laser is designed to be simple and fast. It has a single sail, a single    │
│                 │ hull, and is built for performance and ease of use.                                           │
│                 │                                                                                               │
│                 │ 2. **Size and Dimensions**:                                                                   │
│                 │    - Length: 4.23 meters (13.9 feet)                                                          │
│                 │    - Beam (width): 1.37 meters (4.5 feet)                                                     │
│                 │    - Draft (with the centerboard down): 0.787 meters (2.6 feet)                               │
│                 │                                                                                               │
│                 │ 3. **Sail Area**:                                                                             │
│                 │    - Standard Rig: 7.06 square meters (76 square feet)                                        │
│                 │    - Radial Rig: 5.76 square meters (62 square feet)                                          │
│                 │    - 4.7 Rig: 4.7 square meters (50.6 square feet)                                            │
│                 │                                                                                               │
│                 │ 4. **Rigging**:                                                                               │
│                 │    - The Laser uses a fractional rig with a single mainsail.                                  │
│                 │    - There are different sail sizes (Standard, Radial, and 4.7) that cater to sailors of      │
│                 │ different weights and skill levels.                                                           │
│                 │                                                                                               │
│                 │ 5. **Hull Material**: The hull is made from fiberglass, which provides a good balance between │
│                 │ durability and performance.                                                                   │
│                 │                                                                                               │
│                 │ 6. **Cockpit**: The cockpit is designed to be self-draining and is spacious enough to allow   │
│                 │ the sailor to move and control the boat effectively.                                          │
│                 │                                                                                               │
│                 │ 7. **Portability**: The Laser is relatively lightweight (approximately 59 kilograms or 130    │
│                 │ pounds for the hull), which makes it easier to transport on a car roof rack or trailer.       │
│                 │                                                                                               │
│                 │ 8. **Sailing Category**: It is consi

## Posting to the Coop
The [Coop](https://www.expectedparrot.com) is a new platform for creating, storing and sharing LLM-based research. We can post surveys, agents, results and notebooks, such as this one. [Learn more about using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

In [10]:
from edsl import Notebook

In [11]:
n = Notebook(path = "question_loop_scenarios.ipynb")

In [12]:
n.push(description = "New question method `loop` for creating questions with scenarios", visibility = "public")

{'description': 'New question method `loop` for creating questions with scenarios',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/54bc4f5d-bda5-4c25-9860-0e23af251341',
 'uuid': '54bc4f5d-bda5-4c25-9860-0e23af251341',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [15]:
n = Notebook(path = "question_loop_scenarios.ipynb")

In [16]:
n.patch(uuid = "54bc4f5d-bda5-4c25-9860-0e23af251341", value = n)

{'status': 'success'}